In [ ]:
from google.colab import drive
import nbformat, os

drive.mount('/content/drive', force_remount=True)
SRC = '/content/drive/MyDrive/Colab Notebooks/Untitled15.ipynb'
DST = '/content/drive/MyDrive/colab_saves/cointegration.ipynb'

with open(SRC, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)


nb.metadata.pop('widgets', None)
for cell in nb.cells:
    cell.metadata.pop('widgets', None)
os.makedirs(os.path.dirname(DST), exist_ok=True)
with open(DST, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print("✅ Clean copy saved to", DST)

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

pair = ("SPY", "IVV")
df = yf.download(
    tickers=list(pair),
    start="2025-09-04",
    end="2025-09-09",
    interval="1m",
    progress=False)["Close"].dropna()

prices = pd.DataFrame({
    pair[0]: df[pair[0]],
    pair[1]: df[pair[1]]}).dropna()

prices["log_ratio"] = np.log(prices[pair[0]] / prices[pair[1]])

print(prices.head(), "\n", prices.tail())


/tmp/ipython-input-2397418934.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(


                                  SPY         IVV  log_ratio
Datetime                                                    
2025-09-04 13:30:00+00:00  644.340027  647.479980  -0.004861
2025-09-04 13:31:00+00:00  644.489990  647.650024  -0.004891
2025-09-04 13:32:00+00:00  644.489990  647.659973  -0.004907
2025-09-04 13:33:00+00:00  644.599976  647.799988  -0.004952
2025-09-04 13:34:00+00:00  644.910583  648.130005  -0.004980 
                                   SPY         IVV  log_ratio
Datetime                                                    
2025-09-08 19:55:00+00:00  648.635010  651.869995  -0.004975
2025-09-08 19:56:00+00:00  648.580078  651.820007  -0.004983
2025-09-08 19:57:00+00:00  648.659973  651.900024  -0.004983
2025-09-08 19:58:00+00:00  648.405029  651.640015  -0.004977
2025-09-08 19:59:00+00:00  648.799988  652.049988  -0.004997


In [ ]:
from statsmodels.tsa.stattools import adfuller

# Run ADF test on the log ratio
result = adfuller(prices["log_ratio"])

print("ADF Statistic:", result[0])
print("p-value:", result[1])
print("Lags Used:", result[2])
print("Number of Observations:", result[3])
print("Critical Values:", result[4])


ADF Statistic: -17.494000793750264
p-value: 4.4214198905331885e-30
Lags Used: 3
Number of Observations: 1164
Critical Values: {'1%': np.float64(-3.4359803948357723), '5%': np.float64(-2.8640262259528595), '10%': np.float64(-2.5680937227063922)}


In [ ]:
log_ratios = prices["log_ratio"].values
n = len(log_ratios)
z_scores = [np.nan] * n
initial_capital = 1000
spy_holding = 0
ivv_holding = 0


for t in range(1000, n):
    p_spy = prices["SPY"].iloc[t]
    p_ivv = prices["IVV"].iloc[t]
    mu_t = np.mean(log_ratios[:t+1])
    sigma_t = np.std(log_ratios[:t+1])
    if sigma_t == 0:
      continue
    z_scores[t] = (log_ratios[t] - mu_t) / sigma_t


    if z_scores[t] > 1.25 and spy_holding > -5 :
      initial_capital += p_spy - p_ivv
      spy_holding -= 1
      ivv_holding += 1
      print("short spy and long ivv")

    if z_scores[t] < -1.25 and ivv_holding > -5 :
      initial_capital += p_ivv - p_spy
      spy_holding += 1
      ivv_holding -= 1
      print("short ivv and long spy")

    if z_scores[t] < 0.75:
      if spy_holding < 0 and ivv_holding > 0 :
        initial_capital -= -spy_holding* p_spy
        initial_capital += ivv_holding* p_ivv
        spy_holding = 0
        ivv_holding = 0
        print("close trading")


    if z_scores[t] > -0.5:
      if spy_holding > 0 and ivv_holding < 0:
        initial_capital -= -ivv_holding* p_ivv
        initial_capital += spy_holding* p_spy
        spy_holding = 0
        ivv_holding = 0
        print("close trading")



final_value = (initial_capital + spy_holding * prices["SPY"].iloc[-1]
               + ivv_holding * prices["IVV"].iloc[-1])
print(final_value)



short ivv and long spy
short spy and long ivv
short ivv and long spy
close trading
short spy and long ivv
short ivv and long spy
short ivv and long spy
close trading
short ivv and long spy
close trading
short ivv and long spy
close trading
short spy and long ivv
close trading
short ivv and long spy
close trading
1000.7232666015625


In [ ]:
start = "2025-09-04"
end   = "2025-09-09"
interval = "1m"


pair_1 = ("QQQ", "QQQM")
df_1 = yf.download(tickers=list(pair_1), start=start, end=end, interval=interval, progress=False)["Close"].dropna()
prices_qqq_qqqm = pd.DataFrame({
    pair_1[0]: df_1[pair_1[0]],
    pair_1[1]: df_1[pair_1[1]]
}).dropna()
prices_qqq_qqqm["log_ratio"] = np.log(prices_qqq_qqqm[pair_1[0]] / prices_qqq_qqqm[pair_1[1]])


pair_2 = ("GLD", "IAU")
df_2 = yf.download(tickers=list(pair_2), start=start, end=end, interval=interval, progress=False)["Close"].dropna()
prices_gld_iau = pd.DataFrame({
    pair_2[0]: df_2[pair_2[0]],
    pair_2[1]: df_2[pair_2[1]]
}).dropna()
prices_gld_iau["log_ratio"] = np.log(prices_gld_iau[pair_2[0]] / prices_gld_iau[pair_2[1]])



pair_3 = ("IWM", "VTWO")
df_3 = yf.download(tickers=list(pair_3), start=start, end=end, interval=interval, progress=False)["Close"].dropna()
prices_iwm_vtwo = pd.DataFrame({
    pair_3[0]: df_3[pair_3[0]],
    pair_3[1]: df_3[pair_3[1]]
}).dropna()
prices_iwm_vtwo["log_ratio"] = np.log(prices_iwm_vtwo[pair_3[0]] / prices_iwm_vtwo[pair_3[1]])



/tmp/ipython-input-1382899939.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_1 = yf.download(tickers=list(pair_1), start=start, end=end, interval=interval, progress=False)["Close"].dropna()
/tmp/ipython-input-1382899939.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_2 = yf.download(tickers=list(pair_2), start=start, end=end, interval=interval, progress=False)["Close"].dropna()
/tmp/ipython-input-1382899939.py:29: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_3 = yf.download(tickers=list(pair_3), start=start, end=end, interval=interval, progress=False)["Close"].dropna()


In [ ]:


def adf_test(series, name=""):
    result = adfuller(series.dropna())
    print(f"ADF Test for {name}")
    print(f"  Test Statistic : {result[0]}")
    print(f"  p-value        : {result[1]}")
    for key, value in result[4].items():
        print(f"  Critical Value {key} : {value}")
    if result[1] < 0.05:
        print("  => Stationary (reject H0)\n")
    else:
        print("  => Non-stationary (fail to reject H0)\n")

adf_test(prices_qqq_qqqm["log_ratio"], "QQQ/QQQM")
adf_test(prices_gld_iau["log_ratio"], "GLD/IAU")
adf_test(prices_iwm_vtwo["log_ratio"], "IWM/VTWO")


ADF Test for QQQ/QQQM
  Test Statistic : -18.67468691727232
  p-value        : 2.0436454526974035e-30
  Critical Value 1% : -3.436044130146462
  Critical Value 5% : -2.864054342974637
  Critical Value 10% : -2.568108697208109
  => Stationary (reject H0)

ADF Test for AGG/BND
  Test Statistic : -5.661701960161714
  p-value        : 9.347745631003382e-07
  Critical Value 1% : -3.436014535478888
  Critical Value 5% : -2.8640412872730208
  Critical Value 10% : -2.56810174401953
  => Stationary (reject H0)

ADF Test for IWM/VTWO
  Test Statistic : -3.9159181136485386
  p-value        : 0.0019207721238900157
  Critical Value 1% : -3.436834649927693
  Critical Value 5% : -2.864403033735098
  Critical Value 10% : -2.568294409927983
  => Stationary (reject H0)



In [ ]:


def run_pair_strategy(prices, t1, t2,
                      start_idx=1000,
                      a=1.25, b=1.25, c=0.75, d=0.5,
                      max_units=5,
                      initial_capital_start=1000.0):

    log_ratios = prices["log_ratio"].values
    n = len(log_ratios)
    z_scores = [np.nan] * n
    initial_capital = initial_capital_start
    t1_holding = 0
    t2_holding = 0

    for t in range(start_idx, n):
        p_t1 = prices[t1].iloc[t]
        p_t2 = prices[t2].iloc[t]
        mu_t = np.mean(log_ratios[:t+1])
        sigma_t = np.std(log_ratios[:t+1])
        if sigma_t == 0:
            continue
        z_scores[t] = (log_ratios[t] - mu_t) / sigma_t

        if z_scores[t] > a and t1_holding > -max_units:
            initial_capital += p_t1 - p_t2
            t1_holding -= 1
            t2_holding += 1


        if z_scores[t] < -b and t2_holding > -max_units:
            initial_capital += p_t2 - p_t1
            t1_holding += 1
            t2_holding -= 1

        if z_scores[t] < c:
            if t1_holding < 0 and t2_holding > 0:
                initial_capital -= -t1_holding * p_t1
                initial_capital +=  t2_holding * p_t2
                t1_holding = 0
                t2_holding = 0

        if z_scores[t] > -d:
            if t1_holding > 0 and t2_holding < 0:
                initial_capital -= -t2_holding * p_t2
                initial_capital +=  t1_holding * p_t1
                t1_holding = 0
                t2_holding = 0


    final_value = (initial_capital
                   + t1_holding * prices[t1].iloc[-1]
                   + t2_holding * prices[t2].iloc[-1])
    return final_value



In [ ]:
print(run_pair_strategy(prices_qqq_qqqm, t1="QQQ", t2="QQQM"))
print(run_pair_strategy(prices_gld_iau, t1="GLD", t2="IAU"))
print(run_pair_strategy(prices_iwm_vtwo, t1="IWM", t2="VTWO"))


1000.6388549804688
1000.3001098632812
1000.0300064086914
